### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from astropy.io import fits
from astropy.table import Table
import pyds9
from tabulate import tabulate
from survey_tools import catalog, match
from survey_tools.utility import plot, table

### Options

In [ ]:
catalog_num = 1

match catalog_num:
    case 1:
        catalog_name = '3D-HST'
        field_name   = 'COSMOS' # AEGIS, COSMOS, GOODS-N, GOODS-S, UDS
        filter_name  = 'F160W'  # F125W, F140W, F160W
    case 2:
        catalog_name = 'UVISTA'
        field_name   = None
        filtcatalog.get_paramsn
    case 3:
        catalog_name = 'ZCOSMOS-BRIGHT'
        field_name   = None
        filter_name  = None
    case 4:
        catalog_name = 'ZCOSMOS-DEEP'
        field_name   = None
        filter_name  = None
    case 5:
        catalog_name = 'VUDS'
        field_name   = 'COSMOS' # COSMOS, GOODS-S
        filter_name  = None
    case 6:
        catalog_name = 'Casey'
        field_name   = 'COSMOS' # COSMOS, GOODS-N, UDS
        filter_name  = None
    case 7:
        catalog_name = 'DEIMOS'
        field_name   = None
        filter_name  = None
    case 8:
        catalog_name = 'MOSDEF'
        field_name   = 'COSMOS' # AEGIS, COSMOS, GOODS-N, GOODS-S, UDS
        filter_name  = None
    case 9:
        catalog_name = 'FMOS'
        field_name   = None
        filter_name  = None
    case 10:
        catalog_name = 'KMOS3D'
        field_name   = 'COSMOS' # COSMOS, GOODS-S, UDS
        filter_name  = None
    case 11:
        catalog_name = 'C3R2'
        field_name   = None
        filter_name  = None
    case 12:
        catalog_name = 'LEGAC'
        field_name   = None
        filter_name  = None
    case 13:
        catalog_name = 'HSCSSP'
        field_name   = 'COSMOS' # AEGIS, COSMOS
        filter_name  = None
    case 14:
        catalog_name = 'DESI'
        field_name   = None
        filter_name  = None

catalog_params = catalog.get_params(catalog_name, field_name, filter_name)

### Read Catalogs

In [ ]:
catalog_data = catalog.CatalogData(catalog_params)

### Photometric Catalog Fields

In [ ]:
if table.has_field(catalog_data, 'sources'):
    if isinstance(catalog_data.sources, fits.fitsrec.FITS_rec):
        column_info = np.array([(lambda c: [c.name, c.dtype])(col) for col in catalog_data.sources.columns])
        display(tabulate(column_info, headers=['Name', 'Type'], tablefmt='html'))
        del column_info
    if isinstance(catalog_data.sources, Table):
        column_info = np.array([(lambda c: [c.name, c.dtype])(catalog_data.sources[col]) for col in catalog_data.sources.columns])
        display(tabulate(column_info, headers=['Name', 'Type'], tablefmt='html'))
        del column_info

### Stellar Population Parameters Fields

In [ ]:
if table.has_field(catalog_data, 'spp') and not catalog_data.spp_is_copy:
    if isinstance(catalog_data.spp, fits.fitsrec.FITS_rec):
        column_info = np.array([(lambda c: [c.name, c.dtype])(col) for col in catalog_data.spp.columns])
        display(tabulate(column_info, headers=['Name', 'Type'], tablefmt='html'))
        del column_info
    if isinstance(catalog_data.spp, Table):
        column_info = np.array([(lambda c: [c.name, c.dtype])(catalog_data.spp[col]) for col in catalog_data.spp.columns])
        display(tabulate(column_info, headers=['Name', 'Type'], tablefmt='html'))
        del column_info

### Redshift Catalog Fields

In [ ]:
if table.has_field(catalog_data, 'redshift') and not catalog_data.redshift_is_copy:
    if isinstance(catalog_data.redshift, fits.fitsrec.FITS_rec):
        column_info = np.array([(lambda c: [c.name, c.dtype])(col) for col in catalog_data.redshift.columns])
        display(tabulate(column_info, headers=['Name', 'Type'], tablefmt='html'))
        del column_info
    if isinstance(catalog_data.redshift, Table):
        column_info = np.array([(lambda c: [c.name, c.dtype])(catalog_data.redshift[col]) for col in catalog_data.redshift.columns])
        display(tabulate(column_info, headers=['Name', 'Type'], tablefmt='html'))
        del column_info

### Emission Line Catalog Fields

In [ ]:
if table.has_field(catalog_data, 'lines') and not catalog_data.lines_is_copy:
    if isinstance(catalog_data.lines, fits.fitsrec.FITS_rec):
        column_info = np.array([(lambda c: [c.name, c.dtype])(col) for col in catalog_data.lines.columns])
        display(tabulate(column_info, headers=['Name', 'Type'], tablefmt='html'))
        del column_info

### Catalog Image

In [ ]:
if table.has_field(catalog_params, 'catalog_image_file'):
    image_hdul = catalog.open_image(catalog_params)

    try:
        ds9 = pyds9.DS9(start=True)
    except ValueError:
        ds9 = None
        print('DS9 is not running')

    if ds9 is not None:
        plot.DS9_plot_image(image_hdul)

    image_hdul.close()

### Consolidate Best Data

In [ ]:
catalog.consolidate_best_data(catalog_data)

num_galaxies = len(catalog_data.sources)
print(f"All Sources: {num_galaxies}")

display(tabulate(catalog_data.best[0:10], headers=catalog_data.best.colnames, tablefmt='html'))

### Flatten Galaxy Data

In [ ]:
galaxy_data = catalog.flatten_galaxy_data(catalog_data)

num_galaxies = len(galaxy_data)
print(f"Galaxies: {num_galaxies}")

display(tabulate(galaxy_data[0:10], headers=galaxy_data.colnames, tablefmt='html'))